<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/cody_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpu demo for cody

In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu11 cudf-cu11 cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11

In [2]:
import cuml,cudf
cuml.__version__

'23.04.01'

In [ ]:
!pip install -U --force git+https://github.com/graphistry/pygraphistry.git@cudf
!pip install -U git+https://github.com/graphistry/cu-cat.git@DT

In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import cudf
from time import time
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth', 200)

In [6]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key

# graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='***') ## key id, secret key
graphistry.__version__

import cu_cat
print(cu_cat.__file__)

/usr/local/lib/python3.10/dist-packages/cu_cat/__init__.py


In [7]:
# !nvidia-smi --query-gpu=gpu_name --format=csv
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader
# !nvidia-smi -L
# import sys
# import os

# gpu=(sys.argv[1])
# gpu
import os

Tesla T4


In [8]:
!nvidia-smi --query-gpu=gpu_name --format=csv
!pip install nvsmi
import nvsmi
gpu=str(nvsmi.get_gpu_processes()).split('|')[3].split(':')[1]

name
Tesla T4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://www.dropbox.com/s/owzp6af8v4vcp1s/sample-state-with-severity.csv.zip
!unzip sample-state-with-severity.csv.zip

In [14]:
filtered_df=pd.read_csv("sample-state-with-severity.csv").drop(columns=["Unnamed: 0"])
samp=filtered_df.sample(500000)

from graphistry.features import ModelDict, topic_model

X = ['DRUG_NAME',"severity_level",'BUYER_BUS_ACT',"BUYER_COUNTY","BUYER_STATE"] #buyer_state and county only for multi states

# samp['feats'] = samp[X].astype(str).agg(' '.join, axis=1)
samp['month_date']=pd.to_datetime(samp['month_date'])
samp['day'] = samp['month_date'].dt.day
samp['month'] = samp['month_date'].dt.month
samp['year'] = samp['month_date'].dt.year

samp.drop(columns=['month_date','ds'],inplace=True)

g = graphistry.nodes(cudf.from_pandas(samp))

process=True
if process:
    t=time()
    g2 = g.umap(X=X, feature_engine='cu_cat', engine='cuml')
    print(time()-t)
    # g2.save_search_instance('severity-level-25k.umap')
else:
    print('boo')
    # g2 = g.load_search_instance('severity-level-25k.umap')

g2.plot()

Using GPU: cu_cat


-112.72469186782837
